In [1]:
import torch
import torch.nn as nn
import time
import os

## Load model

In [2]:
class FCN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FCN, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out) 
        return out

In [3]:
model = FCN(1,1,1)
model = torch.load('full_model.pth')
model

FCN(
  (l1): Linear(in_features=36, out_features=25, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=25, out_features=5, bias=True)
)

In [4]:
for weight in model.state_dict():
    print(weight)

l1.weight
l1.bias
l2.weight
l2.bias


In [5]:
for key, val in model.state_dict().items():
    print(key, val.shape)

l1.weight torch.Size([25, 36])
l1.bias torch.Size([25])
l2.weight torch.Size([5, 25])
l2.bias torch.Size([5])


## Print weights for HLS

In [6]:
l1_weight = model.state_dict()['l1.weight'].tolist()
l2_weight = model.state_dict()['l2.weight'].tolist()
l1_bias = model.state_dict()['l1.bias'].tolist()
l2_bias = model.state_dict()['l2.bias'].tolist()

In [7]:
print("const float l1_bias[HIDDEN_SIZE] = {", end='')
biases = ''
for b in l1_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l1_bias[HIDDEN_SIZE] = {0.014873608946800232, 0.10168477147817612, -5.659701824188232, -0.2136642187833786, -0.08962297439575195, -0.19656336307525635, -0.1701306253671646, -0.011637643910944462, 0.032339952886104584, -0.12118472903966904, -0.1856791377067566, -0.10536383092403412, 0.18785347044467926, -0.13703784346580505, -0.1826033741235733, -0.3558380603790283, 0.2419252246618271, -0.12454453110694885, -0.028412925079464912, 1.4173842668533325, 0.10487010329961777, 7.995884895324707, 0.44009652733802795, 1.3006823062896729, -5.884320259094238};


In [8]:
print("const float l2_bias[NUM_CLASSES] = {", end='')
biases = ''
for b in l2_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l2_bias[NUM_CLASSES] = {0.19469130039215088, -3.6987087726593018, -3.60075044631958, -1.8589496612548828, 5.3503241539001465};


In [9]:
print("const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {", end='')
weights = ''
for row in l1_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {{-0.02538423240184784, 0.047698959708213806, 0.009806200861930847, 0.08706597983837128, 0.06885336339473724, -0.11820298433303833, -0.06847713887691498, 0.016615286469459534, 0.021151915192604065, 0.14907671511173248, -0.10695241391658783, -0.09096866846084595, -0.027103766798973083, -0.09533785283565521, 0.04560808837413788, -0.011310979723930359, 0.14390192925930023, 0.11019738018512726, 0.03162364661693573, -0.08531801402568817, -0.06330245733261108, -0.049831390380859375, -0.0937684178352356, 0.020173370838165283, -0.009187102317810059, 0.07121588289737701, 0.058397769927978516, 0.12358833849430084, -0.15658754110336304, -0.09048911184072495, -0.10195708274841309, 0.10330893099308014, -0.10384081304073334, -0.14627967774868011, 0.15487785637378693, 0.007486358284950256},
{-0.23811885714530945, 0.11291610449552536, -0.07249242067337036, 0.03740161657333374, 0.10091887414455414, -0.022262701764702797, -0.057225003838539124, 0.0237930

In [10]:
print("const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {", end='')
weights = ''
for row in l2_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {{-0.04735498130321503, -0.0012612028513103724, -0.2286410629749298, 0.007366053760051727, 0.06823617219924927, 0.21918785572052002, -0.16134904325008392, -0.20922735333442688, -0.19636721909046173, 0.15310898423194885, -0.05080300569534302, -0.08376002311706543, -0.21143504977226257, -0.03812749311327934, -0.13675738871097565, -0.06824340671300888, 0.10982633382081985, -0.06889624893665314, -0.07507294416427612, 0.07896802574396133, 0.06994890421628952, 0.322905570268631, 0.031418174505233765, 0.40636587142944336, -0.054065052419900894},
{0.09160842001438141, -0.049618303775787354, 0.2970571219921112, -0.3541550636291504, 0.1856066733598709, 0.22449007630348206, -0.14453603327274323, 0.02996976301074028, 0.11799098551273346, -0.3010777533054352, -0.24910491704940796, -0.12495382130146027, 0.06615784764289856, -0.05570568889379501, 0.046520400792360306, 0.1632198691368103, 0.06592036038637161, 0.08513083308935165, 0.05517816171050072, 

## Get test data for software-hardware comparison

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing

In [12]:
DATA_DIR = 'data'
RAW = DATA_DIR + '/raw'
NUM_FEATURES = 36

In [13]:
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
X_test = test_df.drop('action', axis='columns')
y_test = test_df['action']

In [14]:
data = [torch.Tensor(X_test.iloc[x].astype(float)) for x in range(5)]

In [15]:
data

[tensor([ 3.0000e-01, -2.7000e-01, -1.1250e-02,  8.9056e-02,  5.2500e-02,
          5.6688e-02,  2.3000e-01, -7.5000e-01,  1.2125e-01,  2.1601e-01,
          6.2500e-02,  1.1950e-01,  6.1000e-01, -2.6000e-01, -1.2075e-01,
          1.7641e-01,  9.2500e-02,  1.0891e-01,  4.3430e+01, -2.4543e+02,
         -5.9545e+00,  4.6427e+01,  1.5230e+01,  2.2302e+01,  1.6134e+02,
         -8.1340e+01,  2.2636e+01,  5.8237e+01,  6.2785e+01,  4.5288e+01,
          5.6100e+01, -6.5770e+01,  2.7320e+00,  2.4537e+01,  1.9565e+01,
          1.7538e+01]),
 tensor([ 1.0600e+00, -1.0900e+00,  1.4800e-01,  4.8760e-01,  5.8000e-01,
          3.8930e-01,  1.1900e+00, -1.0700e+00,  3.0600e-01,  5.1330e-01,
          6.6250e-01,  4.3030e-01,  2.3700e+00, -1.9700e+00,  1.5575e-01,
          1.1012e+00,  1.4750e+00,  8.4325e-01,  2.2483e+02, -3.2075e+02,
         -2.7314e+01,  1.4894e+02,  9.4147e+01,  1.0706e+02,  1.4925e+02,
         -2.7227e+02,  3.6943e+00,  9.5077e+01,  4.1140e+01,  5.8741e+01,
          2.74

In [16]:
with torch.no_grad():
    for d in data:
        start_time = time.time()
        output = model(d)
        action = np.argmax(output)
#         print(f"Inference time: {time.time() - start_time}")    
        print(action)

tensor(0)
tensor(2)
tensor(4)
tensor(4)
tensor(1)


In [17]:
for x in range(5):
    print(y_test[x])

2
2
4
4
1


## Compare software-hardware running time and accuracy

In [18]:
# 1 sample

with torch.no_grad():
    start_time = time.time()

    data = torch.Tensor(X_test.iloc[0].astype(float))
    output = model(data)
    action = np.argmax(output)

    end_time = time.time()
    print(f"Prediction: {action}; Ground truth value: {y_test[0]}")
    print(f"Inference time: {((end_time - start_time) * 1000):.4f} ms") 

Prediction: 0; Ground truth value: 2
Inference time: 1.1930 ms


In [19]:
# whole test set (or dev + test set used in training)

with torch.no_grad():
    start_time = time.time()
    
    num_correct = 0
    num_samples = 0

    for i in range(len(X_test[:])):
        data = torch.Tensor(X_test.iloc[i].astype(float))
        output = model(data)
        action = np.argmax(output)
        num_samples += 1
        num_correct += (action == y_test[i])

    end_time = time.time()
    print(f"Accuracy: {(100.0 * num_correct / num_samples):.2f}%")
    print(f"Inference time: {(end_time - start_time):.4f} s")    

Accuracy: 95.86%
Inference time: 0.0786 s


In [20]:
len(X_test)

169